In [1]:
# Импорт библиотек
import re  # Для работы с регулярными выражениями
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Чтение датасета
df = pd.read_csv('data/data.csv', sep=';')

In [3]:
display(df)

,ID истории болезни,Осн. диаг. при выписке МКБ10 (текст),Заголовок документа,Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Флаг нормы кол. показателя,Кач. лаб. показатель,Значение кач. показателя,Норма кач. показателя,Пол,Дата рождения пациента
0,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Лейкоциты (WBC),1.80,10*9/л,4:11,Пониж,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
1,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Эритроциты (RBC),4.18,10*12/л,"3,8:5,7",Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
2,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Гемоглобин (HGB),120.00,г/л,117:160,Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
3,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Гематокрит (HCT),35.10,%,35:47,Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
4,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Средний объем эритроцита (MCV),84.00,фл,80:99,Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36292,516f652e-8bb8-11eb-bb95-2cea7fe73f75,Острый лейкоз неуточненного клеточного типа,Общий анализ крови,Базофилы,1.00,%,0:2,Норм,Комментарий,Тромбоциты проверены. Рекомендована консультац...,_,М,2014-10-17 00:00:00
36293,516f652e-8bb8-11eb-bb95-2cea7fe73f75,Острый лейкоз неуточненного клеточного типа,Общий анализ крови,Эозинофилы,0.00,%,0:5,Норм,Комментарий,Тромбоциты проверены. Рекомендована консультац...,_,М,2014-10-17 00:00:00
36294,516f652e-8bb8-11eb-bb95-2cea7fe73f75,Острый лейкоз неуточненного клеточного типа,Общий анализ крови,Пролимфоциты,0.00,%,0:0,Норм,Комментарий,Тромбоциты проверены. Рекомендована консультац...,_,М,2014-10-17 00:00:00
36295,516f652e-8bb8-11eb-bb95-2cea7fe73f75,Острый лейкоз неуточненного клеточного типа,Общий анализ крови,Лимфоциты,65.00,%,18:44,Повыш,Комментарий,Тромбоциты проверены. Рекомендована консультац...,_,М,2014-10-17 00:00:00


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36297 entries, 0 to 36296
Data columns (total 13 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   ID истории болезни                    36297 non-null  object 
 1   Осн. диаг. при выписке МКБ10 (текст)  36297 non-null  object 
 2   Заголовок документа                   36297 non-null  object 
 3   Кол. лаб. показатель                  36236 non-null  object 
 4   Значение кол. показателя              36236 non-null  float64
 5   Ед. изм. кол. показателя              36236 non-null  object 
 6   Норма кол. показателя                 36236 non-null  object 
 7   Флаг нормы кол. показателя            36236 non-null  object 
 8   Кач. лаб. показатель                  7005 non-null   object 
 9   Значение кач. показателя              7005 non-null   object 
 10  Норма кач. показателя                 7005 non-null   object 
 11  Пол            

In [5]:
# Удаляем неинформативные столбцы (и так понятно что у нас документ -- ОАК)
print(f'Всего уникальных значений в Заголовке документа: {df["Заголовок документа"].nunique()}')
df = df.drop('Заголовок документа', axis=1)

Всего уникальных значений в Заголовке документа: 1


### Этап 1: Перекодировка пола в числовой признак

(перекодируем значение пола и добавим обработку неожиданных значений)

In [6]:
# Проверяем уникальные значения в столбце "Пол", чтобы убедиться в наличии только ожидаемых значений
unique_gender_values = df['Пол'].unique()
print(f"Уникальные значения в столбце 'Пол' перед преобразованием: {unique_gender_values}")

Уникальные значения в столбце 'Пол' перед преобразованием: ['Ж' 'М']


In [7]:
# Проверяем и обрабатываем значения
if set(unique_gender_values) <= {"Ж", "М"}:
    # Заменяем "Ж" на 0 и "М" на 1
    df['Пол'] = df['Пол'].map({"Ж": 0, "М": 1})
    print("Значения в столбце 'Пол' успешно заменены на числовые коды.")
else:
    # Если найдены другие значения, выводим предупреждение и не изменяем данные
    print("Предупреждение: найдены неожиданные значения в столбце 'Пол'. Проверьте данные перед продолжением.")

Значения в столбце 'Пол' успешно заменены на числовые коды.


### Этап 2: Расчет возраста пациентов



In [8]:
# Преобразование в формат даты с обработкой некорректных значений
df['Дата рождения пациента'] = pd.to_datetime(df['Дата рождения пациента'], errors='coerce')
print("Столбец 'Дата рождения пациента' успешно преобразован в формат даты.")

Столбец 'Дата рождения пациента' успешно преобразован в формат даты.


In [9]:
# Удаляем строки с некорректными датами рождения, если такие имеются
initial_row_count = df.shape[0]
df = df.dropna(subset=['Дата рождения пациента'])
dropped_rows = initial_row_count - df.shape[0]
print(f"Количество удаленных строк с некорректными датами рождения: {dropped_rows}")

Количество удаленных строк с некорректными датами рождения: 0


In [10]:
# Расчет возраста пациента
today = datetime.today()
df['Возраст'] = df['Дата рождения пациента'].apply(
    lambda birth_date: today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
)

In [11]:
# Удаление столбца "Дата рождения пациента"
df = df.drop(columns=['Дата рождения пациента'])
print("Столбец 'Дата рождения пациента' удален из данных после расчета возраста.")

Столбец 'Дата рождения пациента' удален из данных после расчета возраста.


### Этап 3: Обработка пропусков в столбце "Кол. лаб. показатель"



In [12]:
# Выборка с пропущенными значениями по столбцу Кол. лаб. показатель
df_na = df[df["Кол. лаб. показатель"].isna()]
print(f'Всего строк с пропусками по показателям крови: {df_na.shape[0]}')
print(f'Комментарии: {df_na["Кач. лаб. показатель"].unique()}')
print(f'Причины в комментариях: {df_na["Значение кач. показателя"].unique()}')

Всего строк с пропусками по показателям крови: 61
Комментарии: ['Комментарий']
Причины в комментариях: ['сгусток крови!' 'Сгусток' 'сгусток крови'
 'тромбоциты по мазку 81х10*9/л' ' тромбоциты по мазку 23*10*9/л'
 'тромбоциты по мазку 21' 'формулу подсчитать невозможно'
 'тромбоциты пртоверены' 'встречаются микросгустки тромбоцитов' 'сгусток'
 'тромбоциты проверены по мазку'
 'шизоциты 4:1000 эритроцитов(№до 10:1000) ' 'нет на месте'
 'тромбоциты при подсчете по мазку 102' ',' 'ТРОМЮОЦИТЫ ПРОВЕРЕНЫ'
 'тромбоциты проверены' 'Тромбоциты проверены.' 'сгусток в крови'
 'тромбоциты по мазку 64*10*9/Л' 'Тромбоциты проверены\t' 'тромб.пров'
 'СОЭ - 7 мм/ч'
 'Лейкоциты после пересчета (без учета нормобластов) - 14,4х10х9/л. Тромбоциты проверены.  Моноцитоидные элементы отличаются полимрфизмом ядер,  наличием зернистости в цитоплазме , высоким ядерноцитоплазматическим соотношением .'
 'подсчитано 50 клеток, моноцитоидные элементы -16'
 'моноцитарные элементы с нуклеолами в ядре 63,5'
 'Тромбоци

In [13]:
# Определяем значения, при которых строки должны быть удалены
values_to_delete = [
    'сгусток крови!', 'Сгусток', 'сгусток крови', 'формулу подсчитать невозможно',
    'тромбоциты пртоверены', 'встречаются микросгустки тромбоцитов', 'сгусток',
    'тромбоциты проверены по мазку', 'нет на месте', 'сгусток в крови', ',',
    'ТРОМЮОЦИТЫ ПРОВЕРЕНЫ', 'тромбоциты проверены', 'Тромбоциты проверены.', 'Тромбоциты проверены\t', 'тромб.пров',
    'Тромбоциты проверены. Большинство бластных клеток представляет собой клетки крупных и средних размеров, с высоким ядерно-цитоплазматическим соотношением, уродливыми ядрами (неправильной формы ядра). хроматин сглаженный, встречаются нуклеолы. Цитоплазма у части клеток вакуолизированная.',
    'лейкоцитарную формулу подсчитать невозможно', 'подсчет л/ф невозможен',
    'моноцидоидные элементы с грубой структурой хроматина и остатаками нуклеол',
    'Тромбоциты проверены. Подсчет лейкоцитарной формулы невозможен из-за крайне низкого количества лейкоцитов крови.'
]

In [14]:
# Удаляем строки с определенными значениями в "Значение кач. показателя"
initial_row_count = df.shape[0]
df = df[~(df['Кол. лаб. показатель'].isna() & df['Значение кач. показателя'].isin(values_to_delete))]
deleted_rows = initial_row_count - df.shape[0]
print(f"Количество удаленных строк: {deleted_rows}")

Количество удаленных строк: 44


### Этап 4: Объединение одинаковых показателей

Цель: Объединить одинаковые показатели, которые представлены разными наименованиями и единицами, чтобы сделать данные более унифицированными и удобными для анализа.

In [15]:
# Вывод всех уникальных значений в колонке "Кол. лаб. показатель"
unique_lab_indicators = df['Кол. лаб. показатель'].unique()
print("Уникальные значения в колонке 'Кол. лаб. показатель':")
print(unique_lab_indicators)

Уникальные значения в колонке 'Кол. лаб. показатель':
['Лейкоциты (WBC)' 'Эритроциты (RBC)' 'Гемоглобин (HGB)'
 'Гематокрит (HCT)' 'Средний объем эритроцита (MCV)'
 'Среднее содержание гемоглобина в эритроците (MCH)'
 'Средняя концентрация гемоглобина в эритроците (MCHC)' 'Тромбоциты (PLT)'
 'Нейтрофилы, относительное количество (NE%)'
 'Лимфоциты, относительное количество (LY%)'
 'Нейтрофилы, абсолютное количество (NE#)'
 'Лимфоциты, абсолютное количество (LY#)'
 'Смешанная фракция, абсолютное количество (MXD#)'
 'Смешанная фракция, относительное количество (MXD%)'
 'Ширина распределения эритроцитов по объему, коэффициент вариации (RDW-CV)'
 'Ширина распределения эритроцитов по объему, стандартное отклонение (RDW-SD)'
 'Ширина распределения эритроцитов по объему (RDW)' 'Тромбокрит (PCT)'
 'Средний объем тромбоцита (MPV)' 'Ширина распределения тромбоцитов (PDW)'
 'Моноциты, относительное количество (MO%)'
 'Эозинофилы, относительное количество (EO%)'
 'Базофилы, относительное количеств

In [31]:
# Создаем словарь для унификации показателей и приведения их к общему названию
unification_dict = {
    'Лейкоциты (WBC)': 'Лейкоциты (WBC)',
    'WBC': 'Лейкоциты (WBC)',
    'Общее количество лейкоцитов (WBC)': 'Лейкоциты (WBC)',
    'Лейкоциты': 'Лейкоциты (WBC)', # проверено
    
    'Эритроциты (RBC)': 'Эритроциты (RBC)',
    'RBC': 'Эритроциты (RBC)',
    'Общее количество эритроцитов (RBC)': 'Эритроциты (RBC)',
    'Эритроциты': 'Эритроциты (RBC)', # проверено
    
    'Гемоглобин (HGB)': 'Гемоглобин (HGB)',
    'HGB': 'Гемоглобин (HGB)',
    'Гемоглобин': 'Гемоглобин (HGB)', # проверено
    
    'Гематокрит (HCT)': 'Гематокрит (HCT)',
    'HCT': 'Гематокрит (HCT)',
    'Гематокрит': 'Гематокрит (HCT)', # проверено

    'Тромбоциты (PLT)': 'Тромбоциты (PLT)',
    'PLT': 'Тромбоциты (PLT)',
    'Тромбоциты': 'Тромбоциты (PLT)', # проверено
    
    'Средний объем эритроцита (MCV)': 'Средний объем эритроцита (MCV)',
    'MCV': 'Средний объем эритроцита (MCV)',
    'Средний объем эритроцита': 'Средний объем эритроцита (MCV)', # проверено
    
    'Среднее содержание гемоглобина в эритроците (MCH)': 'Среднее содержание гемоглобина в эритроците (MCH)',
    'MCH': 'Среднее содержание гемоглобина в эритроците (MCH)',
    'Среднее содержание гемоглобина в эритроците': 'Среднее содержание гемоглобина в эритроците (MCH)', # проверено
    
    'Средняя концентрация гемоглобина в эритроците (MCHC)': 'Средняя концентрация гемоглобина в эритроците (MCHC)',
    'MCHC': 'Средняя концентрация гемоглобина в эритроците (MCHC)',
    'Средняя концентрация гемоглобина в эритроците': 'Средняя концентрация гемоглобина в эритроците (MCHC)', # проверено

    'Средний объем тромбоцита (MPV)': 'Средний объем тромбоцита (MPV)',
    'MPV': 'Средний объем тромбоцита (MPV)',
    'MPV ': 'Средний объем тромбоцита (MPV)',
    'MPV (Средний объём тромбоцитов)': 'Средний объем тромбоцита (MPV)',
    'Средний объём тромбоцитов': 'Средний объем тромбоцита (MPV)',
    'Средний объём тромбоцита': 'Средний объем тромбоцита (MPV)', # проверено

    'Тромбокрит (PCT)': 'Тромбокрит (PCT)',
    'PCT': 'Тромбокрит (PCT)',
    'Тромбокрит': 'Тромбокрит (PCT)', # проверено
    
    'Ширина распределения эритроцитов по объему (RDW)': 'Распределение эритроцитов по объему (RDW)',
    'Ширина распределения эритроцитов': 'Распределение эритроцитов по объему (RDW)',
    'RDW': 'Распределение эритроцитов по объему (RDW)', # проверено
    'Ширина распределения эритроцитов (RDW)': 'Распределение эритроцитов по объему (RDW)',

    'Ширина распределения эритроцитов по объему, коэффициент вариации (RDW-CV)': 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)',
    'Ширина распределения эритроцитов по объему, коэффициент вариации': 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)',
    'RDW-CV': 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)',
    'RDW-CV ': 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)', # проверено

    'Ширина распределения эритроцитов по объему, стандартное отклонение (RDW-SD)': 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)',
    'Ширина распределения эритроцитов по объему, стандартное отклонение': 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)', 
    'Ширина распределения эритроцитов, стандартное отклонение (RDW-SD)': 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)',
    'RDW-SD': 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)', # проверено

    'Ширина распределения тромбоцитов (PDW)': 'Распределение тромбоцитов по объему (PDW)',
    'Ширина распределения тромбоцитов': 'Распределение тромбоцитов по объему (PDW)',
    'Ширина распределения тромбоцитов по объему': 'Распределение тромбоцитов по объему (PDW)',
    'PDW': 'Распределение тромбоцитов по объему (PDW)', # проверено

    'Скорость оседания эритроцитов (СОЭ) по Вестергрену': 'СОЭ по Вестергрену',
    'СОЭ Вест.': 'СОЭ по Вестергрену',
    # 'СОЭ по Панч.': 'СОЭ', я предлагаю вообще дропнуть эти показатели, т.к.
    # метод Вестергрена одобрен в качестве эталонного Международным комитетом по стандартизации в гематологии (ICSH) 
    # и регулируется стандартом Института клинических и лабораторных стандартов (CLSI), а по Панченкову
    # вроде как считается устаревшим...
    'СОЭ': 'СОЭ по Вестергрену', # проверено
    
    'Нейтрофилы, относительное количество (NE%)': 'Нейтрофилы, относительное количество (NE%)',
    'Относительное количество нейтрофилов (NE%)': 'Нейтрофилы, относительное количество (NE%)',
    'NE%': 'Нейтрофилы, относительное количество (NE%)',
    'Нейтрофилы %': 'Нейтрофилы, относительное количество (NE%)',
    'Нейтрофилы, относительное количество': 'Нейтрофилы, относительное количество (NE%)', # проверено
    
    'Нейтрофилы, абсолютное количество (NE#)': 'Нейтрофилы, абсолютное количество (NE#)',
    'Абсолютное количество нейтрофилов (NE#)': 'Нейтрофилы, абсолютное количество (NE#)',
    'Нейтрофилы #': 'Нейтрофилы, абсолютное количество (NE#)',
    'Нейтрофилы, абсолютное количество': 'Нейтрофилы, абсолютное количество (NE#)', # проверено
    
    'Лимфоциты, относительное количество (LY%)': 'Лимфоциты, относительное количество (LY%)',
    'Относительное количество лимфоцитов (LY%)': 'Лимфоциты, относительное количество (LY%)',
    'LY%': 'Лимфоциты, относительное количество (LY%)',
    'Лимфоциты %': 'Лимфоциты, относительное количество (LY%)',
    'Лимфоциты, относительное количество': 'Лимфоциты, относительное количество (LY%)', # проверено
    
    'Лимфоциты, абсолютное количество (LY#)': 'Лимфоциты, абсолютное количество (LY#)',
    'Абсолютное количество лимфоцитов (LY#)': 'Лимфоциты, абсолютное количество (LY#)',
    'Лимфоциты #': 'Лимфоциты, абсолютное количество (LY#)',
    'Лимфоциты, абсолютное количество': 'Лимфоциты, абсолютное количество (LY#)', # проверено

    'Моноциты, относительное количество (MO%)': 'Моноциты, относительное количество (MO%)',
    'Моноциты, относительное количество': 'Моноциты, относительное количество (MO%)',
    'Относительное количество моноцитов (MO%)': 'Моноциты, относительное количество (MO%)',
    'MO%': 'Моноциты, относительное количество (MO%)',
    'Моноциты %': 'Моноциты, относительное количество (MO%)', # проверено

    'Моноциты, абсолютное количество (MO#)': 'Моноциты, абсолютное количество (MO#)',
    'Моноциты, абсолютное количество': 'Моноциты, абсолютное количество (MO#)',
    'Абсолютное количество моноцитов (MO#)': 'Моноциты, абсолютное количество (MO#)',
    'Моноциты #': 'Моноциты, абсолютное количество (MO#)', # проверено

    'Эозинофилы, относительное количество (EO%)': 'Эозинофилы, относительное количество (EO%)',
    'Эозинофилы, относительное количество': 'Эозинофилы, относительное количество (EO%)',
    'Относительное количество эозинофилов (EO%)': 'Эозинофилы, относительное количество (EO%)',
    'Эозинофилы %': 'Эозинофилы, относительное количество (EO%)',
    'EO%': 'Эозинофилы, относительное количество (EO%)', # проверено

    'Эозинофилы, абсолютное количество (EO#)': 'Эозинофилы, абсолютное количество (EO#)',
    'Эозинофилы, абсолютное количество': 'Эозинофилы, абсолютное количество (EO#)',
    'Абсолютное количество эозинофилов (EO#)': 'Эозинофилы, абсолютное количество (EO#)',
    'Эозинофилы # ': 'Эозинофилы, абсолютное количество (EO#)',
    'Эозинофилы #': 'Эозинофилы, абсолютное количество (EO#)', # проверено

    'Базофилы, относительное количество (BA%)': 'Базофилы, относительное количество (BA%)',
    'Базофилы, относительное количество': 'Базофилы, относительное количество (BA%)',
    'Относительное количество базофилов (BA%)': 'Базофилы, относительное количество (BA%)',
    'Базофилы %': 'Базофилы, относительное количество (BA%)',
    'BA%': 'Базофилы, относительное количество (BA%)', # проверено

    'Базофилы, абсолютное количество (BA#)': 'Базофилы, абсолютное количество (BA#)',
    'Абсолютное количество базофилов (BA#)': 'Базофилы, абсолютное количество (BA#)',
    'Базофилы, абсолютное количество': 'Базофилы, абсолютное количество (BA#)',
    'Базофилы #': 'Базофилы, абсолютное количество (BA#)',
    'Базофилы # ': 'Базофилы, абсолютное количество (BA#)', # проверено
    
    'Смешанная фракция, относительное количество (MXD%)': 'Смешанная фракция, относительное количество (MXD%)',
    'Смешанная фракция, относительное количество': 'Смешанная фракция, относительное количество (MXD%)',
    'MXD%': 'Смешанная фракция, относительное количество (MXD%)', # проверено
        
    'Смешанная фракция, абсолютное количество (MXD#)': 'Смешанная фракция, абсолютное количество (MXD#)',
    'Смешанная фракция, абсолютное количество': 'Смешанная фракция, абсолютное количество (MXD#)',
    'MXD#': 'Смешанная фракция, абсолютное количество (MXD#)',
    'MXD# ': 'Смешанная фракция, абсолютное количество (MXD#)', # проверено

    'Нормобласты %': 'Нормобласты, относительное количество (%)', # проверено
    'Нормобласты #': 'Нормобласты, абсолютное количество (#)', # проверено
    'Ретикулоциты %': 'Ретикулоциты, абсолютное количество (#)', # проверено
    'Ретикулоциты кол-во': 'Ретикулоциты, абсолютное количество (#)', # проверено
    'RET%': 'Ретикулоциты, относительное количество (%)', # проверено
    'Цветовой показатель': 'Цветовой показатель', 
    
    'Палочкоядерные': 'Палочкоядерные нейтрофилы (%)',
    'Сегментоядерные': 'Сегментоядерные нейтрофилы (%)',
    'Базофилы': 'Базофилы (лейкоцитарная формула, %)',
    'Лимфоциты': 'Лимфоциты (лейкоцитарная формула, %)',
    'Моноциты': 'Моноциты (лейкоцитарная формула, %)',
    'Эозинофилы': 'Эозинофилы (лейкоцитарная формула, %)',
    'Плазматические клетки': 'Плазматические клетки',
    'Плазматич. клетки': 'Плазматические клетки',
    'Миелоциты': 'Миелоциты',
    'Нормобласты': 'Нормобласты',
    
    'Пролимфоциты': 'Пролимфоциты',
    'Промиелоциты': 'Промиелоциты',
    'P-LCR': 'P-LCR',
}


In [32]:
# Применяем унификацию названий
df['Кол. лаб. показатель'] = df['Кол. лаб. показатель'].replace(unification_dict)
print("Наименования показателей успешно унифицированы.")

Наименования показателей успешно унифицированы.


In [33]:
new_unique_lab_indicators = df['Кол. лаб. показатель'].unique()
print("Новые уникальные значения в колонке 'Кол. лаб. показатель':")
print(new_unique_lab_indicators)

Новые уникальные значения в колонке 'Кол. лаб. показатель':
['Лейкоциты (WBC)' 'Эритроциты (RBC)' 'Гемоглобин (HGB)'
 'Гематокрит (HCT)' 'Средний объем эритроцита (MCV)'
 'Среднее содержание гемоглобина в эритроците (MCH)'
 'Средняя концентрация гемоглобина в эритроците (MCHC)' 'Тромбоциты (PLT)'
 'Нейтрофилы, относительное количество (NE%)'
 'Лимфоциты, относительное количество (LY%)'
 'Нейтрофилы, абсолютное количество (NE#)'
 'Лимфоциты, абсолютное количество (LY#)'
 'Смешанная фракция, абсолютное количество (MXD#)'
 'Смешанная фракция, относительное количество (MXD%)'
 'Распределение эритроцитов по объему, коэффициент вариации (RDW-CV)'
 'Распределение эритроцитов по объему, стандартное отклонение (RDW-SD)'
 'Распределение эритроцитов по объему (RDW)' 'Тромбокрит (PCT)'
 'Средний объем тромбоцита (MPV)'
 'Распределение тромбоцитов по объему (PDW)'
 'Моноциты, относительное количество (MO%)'
 'Эозинофилы, относительное количество (EO%)'
 'Базофилы, относительное количество (BA%)'
 '

In [34]:
# Список значений для удаления
red_highlighted_values = [
    'Средний объем тромбоцита', 'Юные', 'Бласты', 'Нормобласты', 
    'СОЭ Панч.', 'Неклассифицируемые кол-во',
    'Неклассифицируемые %', 'MXD', 'СОЭ по Панченкову'
]

In [35]:
# Фильтруем строки с показателями, которые выделены красным цветом
filtered_df = df[df['Кол. лаб. показатель'].isin(red_highlighted_values)]

In [36]:
# Считаем количество каждого показателя
count_per_indicator = filtered_df['Кол. лаб. показатель'].value_counts()

# Выводим результаты
print("Количество записей по каждому показателю, выделенному красным цветом:")
print(count_per_indicator)

Количество записей по каждому показателю, выделенному красным цветом:
Series([], Name: count, dtype: int64)


короче дропну я начиная с юных, мне сложно их интерпертировать, а результат может пострадать.... если что отменить можно всегда

In [37]:
# Список значений, которые нужно удалить из столбца "Кол. лаб. показатель"
values_to_remove = [
    'Юные', 'Неклассифицируемые %', 'СОЭ Панч.', 'Нормобласты', 
    'Бласты', 'Неклассифицируемые кол-во', 'MXD', 'Средний объем тромбоцита', 
    'СОЭ по Панченкову'
]

In [38]:
# Удаление строк, где "Кол. лаб. показатель" имеет одно из значений в списке
df = df[~df['Кол. лаб. показатель'].isin(values_to_remove)]

In [39]:
# Проверка результата
print("Строки с указанными значениями удалены.")
print(df['Кол. лаб. показатель'].value_counts())

Строки с указанными значениями удалены.
Кол. лаб. показатель
Гемоглобин (HGB)                                                        1619
Гематокрит (HCT)                                                        1619
Средний объем эритроцита (MCV)                                          1619
Среднее содержание гемоглобина в эритроците (MCH)                       1619
Средняя концентрация гемоглобина в эритроците (MCHC)                    1619
Эритроциты (RBC)                                                        1619
Тромбоциты (PLT)                                                        1617
Лейкоциты (WBC)                                                         1616
Лимфоциты, абсолютное количество (LY#)                                  1470
Лимфоциты, относительное количество (LY%)                               1470
Нейтрофилы, относительное количество (NE%)                              1332
Нейтрофилы, абсолютное количество (NE#)                                 1319
Средний объем т

Думаю удалить также следующие показали по причине того, что их мало в нашем датасете и они мало информативны:
* P-LCR  
* Плазматические клетки 

In [40]:
# Список значений, которые нужно удалить из столбца "Кол. лаб. показатель"
values_to_remove2 = [
    'P-LCR', 'Плазматические клетки',
]

In [41]:
# Удаление строк, где "Кол. лаб. показатель" имеет одно из значений в списке
df = df[~df['Кол. лаб. показатель'].isin(values_to_remove2)]

In [42]:
# Проверка результата
print("Строки с указанными значениями удалены.")
print(df['Кол. лаб. показатель'].value_counts())

Строки с указанными значениями удалены.
Кол. лаб. показатель
Гемоглобин (HGB)                                                        1619
Гематокрит (HCT)                                                        1619
Средний объем эритроцита (MCV)                                          1619
Среднее содержание гемоглобина в эритроците (MCH)                       1619
Средняя концентрация гемоглобина в эритроците (MCHC)                    1619
Эритроциты (RBC)                                                        1619
Тромбоциты (PLT)                                                        1617
Лейкоциты (WBC)                                                         1616
Лимфоциты, абсолютное количество (LY#)                                  1470
Лимфоциты, относительное количество (LY%)                               1470
Нейтрофилы, относительное количество (NE%)                              1332
Нейтрофилы, абсолютное количество (NE#)                                 1319
Средний объем т

## Что дальше: 

1.  нужно понять как переназвать `Миелоциты`, `Промиелоциты`, `Пролимфоциты`. Может (и скорее всего) отнести из в группу "Лейкоцитарная формула", так как они в процентах вроде даны (проверить)

2.  нужно обработать колонки:

   `Ед. изм. кол. показателя` (привести все единицы измерения к стандартным),	
   `Норма кол. показателя` (проверить нормы, привести к стандартным),	
   `Флаг нормы кол. показателя` (тут тоже привести к "Норм", "Пониж", "Повыш"),	
   `Кач. лаб. показатель` (разобраться, можно ли вообще дропнуть этот столбец?),	
   `Значение кач. показателя` (тут остались комментарии, которые содержат полезную инфу для столбца "Значение кол. показателя", нужно ее оттуда достать и поставить на место) ,	
   `Норма кач. показателя` (разобраться, можно ли вообще дропнуть этот столбец?) 

In [49]:
print(df['Ед. изм. кол. показателя'].value_counts())

Ед. изм. кол. показателя
%          14270
10*9/л      8808
фл          3665
г/л         2528
10*12/л     1619
пг          1619
_            885
г/дл         715
мм/час       568
fL           136
FL            59
Name: count, dtype: int64


In [24]:
# Преобразуем единицы измерения для показателя "Гемоглобин" с г/дл в г/л, если такие встречаются
# Например, для показателя "Гемоглобин" единицы г/дл нужно перевести в г/л, умножив на 10
mask_hb = (df['Кол. лаб. показатель'] == 'Гемоглобин') & (df['Ед. изм. кол. показателя'] == 'г/дл')
df.loc[mask_hb, 'Значение кол. показателя'] *= 10
df.loc[mask_hb, 'Ед. изм. кол. показателя'] = 'г/л'
print("Единицы измерения для показателя 'Гемоглобин' успешно преобразованы в г/л.")

Единицы измерения для показателя 'Гемоглобин' успешно преобразованы в г/л.


In [25]:
# Исправляем обозначения единиц для других показателей
# Пример: приводим FL и фл к одному обозначению
df['Ед. изм. кол. показателя'] = df['Ед. изм. кол. показателя'].replace({'FL': 'фл', 'fL': 'фл', 'фЛ': 'фл'})
print("Единицы измерения успешно унифицированы для обозначения 'фл'.")

Единицы измерения успешно унифицированы для обозначения 'фл'.


## Что дальше: 

1. Нужно избавиться от пропусков. Посмотреть где они есть, в каких столбцах, узнать причину (может просто в комментарии указана инфа, но не занесена в нужный столбец) и разобраться с этим.

   Больше всего пропусков кстати в столбцах, которые я предлагаю тупо дропнуть (после вытаскивания от туда полезной инфы, конечно, если такая имеется) 

In [50]:
# Проверяем, какие столбцы содержат пропуски и сколько их
missing_info = df.isnull().sum()
print("Количество пропусков по столбцам перед обработкой:")
print(missing_info[missing_info > 0])

Количество пропусков по столбцам перед обработкой:
Кол. лаб. показатель             17
Значение кол. показателя         17
Ед. изм. кол. показателя         17
Норма кол. показателя            17
Флаг нормы кол. показателя       17
Кач. лаб. показатель          28252
Значение кач. показателя      28252
Норма кач. показателя         28252
dtype: int64


In [51]:
# Фильтруем строки с пропусками в столбце "Кол. лаб. показатель" для изучения данных в других столбцах
missing_lab_indicator_df = df[df["Кол. лаб. показатель"].isna()]
missing_lab_indicator_df

,ID истории болезни,Осн. диаг. при выписке МКБ10 (текст),Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Флаг нормы кол. показателя,Кач. лаб. показатель,Значение кач. показателя,Норма кач. показателя,Пол,Возраст
2378,b36a2106-4308-11ed-ab5a-0050568844e6,Хронический лимфоцитарный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 81х10*9/л,_,1,77
5609,eebeb925-9f8e-11ec-ab54-0050568844e6,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 23*10*9/л,_,1,68
5610,eebeb925-9f8e-11ec-ab54-0050568844e6,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 21,_,1,68
14713,a6cc0de1-ac03-11ec-ab54-0050568844e6,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,шизоциты 4:1000 эритроцитов(№до 10:1000),_,1,66
17789,7b9025cb-ae7c-11ec-ab54-0050568844e6,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты при подсчете по мазку 102,_,0,72
22833,268d2f6a-32ae-11ea-80c5-901b0e633689,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 64*10*9/Л,_,0,86
25849,81f895b9-2ee2-11eb-80e0-901b0e633689,Другой миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,СОЭ - 7 мм/ч,_,1,50
27046,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,Лейкоциты после пересчета (без учета нормоблас...,_,0,61
27047,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,"подсчитано 50 клеток, моноцитоидные элементы -16",_,0,61
27198,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,"моноцитарные элементы с нуклеолами в ядре 63,5",_,0,61


In [33]:
missing_val_indicator_df = df[df["Норма кол. показателя"].isna()]
missing_val_indicator_df

,ID истории болезни,Осн. диаг. при выписке МКБ10 (текст),Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Флаг нормы кол. показателя,Кач. лаб. показатель,Значение кач. показателя,Норма кач. показателя,Пол,Возраст
2378,b36a2106-4308-11ed-ab5a-0050568844e6,Хронический лимфоцитарный лейкоз,Тромбоциты,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 81х10*9/л,_,1,77
5609,eebeb925-9f8e-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Тромбоциты,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 23*10*9/л,_,1,68
5610,eebeb925-9f8e-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Тромбоциты,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 21,_,1,68
14713,a6cc0de1-ac03-11ec-ab54-0050568844e6,Другой уточненный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,шизоциты 4:1000 эритроцитов(№до 10:1000),_,1,66
17789,7b9025cb-ae7c-11ec-ab54-0050568844e6,Другой уточненный лейкоз,Тромбоциты,NaN,NaN,NaN,NaN,Комментарий,тромбоциты при подсчете по мазку 102,_,0,72
22833,268d2f6a-32ae-11ea-80c5-901b0e633689,Другой уточненный лейкоз,Тромбоциты,NaN,NaN,NaN,NaN,Комментарий,тромбоциты по мазку 64*10*9/Л,_,0,86
25849,81f895b9-2ee2-11eb-80e0-901b0e633689,Другой миелоидный лейкоз,СОЭ,7.0,мм/ч,NaN,NaN,Комментарий,СОЭ - 7 мм/ч,_,1,50
27046,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,Тромбоциты,NaN,NaN,NaN,NaN,Комментарий,Лейкоциты после пересчета (без учета нормоблас...,_,0,61
27047,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,"подсчитано 50 клеток, моноцитоидные элементы -16",_,0,61
27198,72f64027-6cf5-11ea-80ce-901b0e63368b,Острый миелоидный лейкоз,NaN,NaN,NaN,NaN,NaN,Комментарий,"моноцитарные элементы с нуклеолами в ядре 63,5",_,0,61
